In [1]:
'''
adjusted: aumentar a fonte, colocar só o grafico rebatido. Colocar uns 6 em subplots.
Escolher 

Means: colocar como último gráfico. 
	Média das medidas adicionar. 
	Colocar como pontos (marker?)

Results:

1 - Splines
2 - Gráfico adjusted - larguras - 6 imagens, incluir as melhores e algumas piores
3 - Gráfico de média, ajustar o gráfico, para a melhor curva.

Foco para essa semana. Adicionar todos os gráficos e escrever um parágrafo sobre eles.

Procurar artigos sobre semi supervisionado / splines para colocar na introd.

Incluir na metodologia: medição das peças
'''

'\nadjusted: aumentar a fonte, colocar só o grafico rebatido. Colocar uns 6 em subplots.\nEscolher \n\nMeans: colocar como último gráfico. \n\tMédia das medidas adicionar. \n\tColocar como pontos (marker?)\n\nResults:\n\n1 - Splines\n2 - Gráfico adjusted - larguras - 6 imagens, incluir as melhores e algumas piores\n3 - Gráfico de média, ajustar o gráfico, para a melhor curva.\n\nFoco para essa semana. Adicionar todos os gráficos e escrever um parágrafo sobre eles.\n\nProcurar artigos sobre semi supervisionado / splines para colocar na introd.\n\nIncluir na metodologia: medição das peças\n'

In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge

In [38]:
sns.set_style('whitegrid')

MEDIUM_SIZE = 20
SIDE_BY_SIDE = 14.5

#Font sizes
plt.rc('xtick', labelsize=MEDIUM_SIZE)
plt.rc('ytick', labelsize=MEDIUM_SIZE)
plt.rc('legend', fontsize=MEDIUM_SIZE)
plt.rc('font', size=MEDIUM_SIZE)

In [39]:
def save_true_vs_pred_plot(eval_results_df, directory, save=False):

    true_vs_pred_plot_name = '\\true_vs_pred_lineplot_NEW.png'
    true_vs_pred_plot_path = directory + true_vs_pred_plot_name

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))

    #Figure size
    
    #fig.set_figwidth(16)
    #fig.set_figheight(8)

    #Create the true vs pred plot
    sns.scatterplot(data=eval_results_df, x='true', y='pred', s=5, color='b', ax=ax1)
    sns.kdeplot(data=eval_results_df, x='true', label='Measures', ax=ax2, color='black')
    sns.kdeplot(data=eval_results_df, x='pred', label='Predicted values', ax=ax2, color='b')

    #Define lower, upper bounds for x and y
    xy_min = min(eval_results_df['true'].min(), eval_results_df['pred'].min())
    xy_max = max(eval_results_df['true'].max(), eval_results_df['pred'].max())

    ax1.set_xlim([xy_min, xy_max])
    ax1.set_ylim([xy_min, xy_max])

    ax1.set_title(label='True measures vs predicted values', fontdict={'fontsize': 11})
    ax2.set_title(label='Distribution density of measures and predictions', fontdict={'fontsize': 11})
    
    ax1.set_xlabel('Measured Values')
    ax1.set_ylabel('Predicted Values')
    ax2.set_xlabel('Width')
    ax2.set_ylabel('Probability Density')

    ax1.legend()
    ax2.legend()

    #Save and show figure
    if save==True:
        fig.savefig(true_vs_pred_plot_path)
        plt.close()
    else:
        plt.show()

In [40]:
def save_mean_true_vs_pred_plot(eval_results_df, directory, save=False):
        standoff_x_plot_name = '\\standoff_x_lineplot_NEW.png'
        standoff_x_plot_path = directory + standoff_x_plot_name

        #Figure size
        plt.figure(figsize=(11,7.5))

        standoff_grouped_true = eval_results_df.groupby('standoff distance')['true', 'pred'].mean()
        sns.scatterplot(data=eval_results_df, x='standoff distance', y='pred', s=3, alpha=0.5, color='b', label='Predicted widths')
        sns.scatterplot(data=standoff_grouped_true, x=standoff_grouped_true.index, y='true', marker='o', s=45, color='black', label='Mean of measured widths for each standoff distance')
        sns.lineplot(data=standoff_grouped_true, x=standoff_grouped_true.index, y='pred', color='g', label='Mean of predicted widths for each standoff distance')
        
        plt.title(f'Predicted widths for each standoff distance against mean of measured values', fontdict={'fontsize': 16})
        plt.legend()
        plt.xlabel('Standoff distance (mm)', fontsize=16)
        plt.ylabel('Width (mm)', fontsize=16)
        
        if save:
            plt.savefig(standoff_x_plot_path)
            plt.close()
        else:
            plt.show()

In [41]:
def save_width_over_x_plot(eval_results_df, directory, adjusted=False, save=False):
        standoff_distances_folder = directory + '\\plots_standoff_distance_NEW'
        try:
            os.mkdir(standoff_distances_folder)
        except:
            pass

        '''for standoff_distance, df in eval_results_df.groupby(by='standoff distance'):
            df.sort_values(by='x pos', inplace=True, ascending=True)
            
            plt.figure(figsize=(14,6.5))
            plt.ylim([0.6, 1.050])
            
            try:
                sns.lineplot(x=np.linspace(0, 136, 136), y=[0]*136)
            except:
                pass

            sns.lineplot(data=df, x='x pos', y='true', color='black', label='True measures')
            sns.lineplot(data=df, x='x pos', y='pred', color='orange', label='Predicted Values')
            
            plt.xticks(rotation=45)
            plt.locator_params(axis='x', nbins=10)
            plt.title(f'Standoff distance: {standoff_distance}', fontdict={'fontsize': 14})
            plt.legend(loc='lower right')
            plt.xlabel('Lenght (mm)', fontsize=14)
            plt.ylabel('Width (mm)', fontsize=14)
            plt.tight_layout()

            #save each file to folder
            curr_standoff_dist_plot_path = f'\\{standoff_distance}.png'
            if save:
                
                plt.savefig(standoff_distances_folder + curr_standoff_dist_plot_path)
                plt.close()
            else:
                plt.show()
                break'''
        
        if adjusted:
            for standoff_distance, df in eval_results_df.groupby(by='standoff distance'):
                df.sort_values(by='x pos', inplace=True, ascending=True)

                #figure configs
                plt.figure(figsize=(12,6))
                #plt.ylim([-550,550])
                plt.ylim([-0.55, 0.55])

                #calculate adjusted y values
                x_pos = df['x pos'].to_numpy()
                true_over = df['true'].to_numpy() / 2
                true_under = df['true'].to_numpy() / -2
                pred_over = df['pred'].to_numpy() / 2
                pred_under = df['pred'].to_numpy() / -2

                #plot data
                sns.lineplot(x=x_pos, y=true_over, color='black', label='Measured values')
                sns.lineplot(x=x_pos, y=pred_over, color='blue', label='Predicted values')
                sns.lineplot(x=x_pos, y=true_under, color='black')
                sns.lineplot(x=x_pos, y=pred_under, color='blue')
                #sns.lineplot(x=np.linspace(0, 135.33, df.shape[0]).astype('int'), y=pred_under, color='orange', label='pred under')

                #plot configs
                plt.locator_params(axis='x', nbins=13)
                plt.legend()
                plt.title(f'Reflected width for standoff distance: {standoff_distance}', fontdict={'fontsize': 20})
                plt.xlabel('Length (mm)', fontsize=20)
                plt.ylabel('Width (mm)', fontsize=20)
                plt.tight_layout()
                
                #save each file to folder
                curr_standoff_dist_plot_path = f'\\scaled_{standoff_distance}.png'
                if save:
                    plt.savefig(standoff_distances_folder + curr_standoff_dist_plot_path)
                    plt.close()
                else:
                    plt.show()
                    break

In [42]:
def plot_spline(degree, knots, x_plot, y_plot, x_true, y_true, label, save=False, show=False):
    #Plot curves
    plt.figure(figsize=(10,8)) # 7, 5 for alone image
    plt.plot(x_plot, y_plot, color='blue', label='Predicted spline')
    sns.scatterplot(x=x_true, y=y_true, color='black', label='Measured points')
    
    plt.xlabel('Length (mm)')
    plt.ylabel('Width (mm)')
    
    plt.title(f'Spline prediction for {degree} degrees and {knots} knots', fontdict={'fontsize': 20}) #14 for alone image
    plt.legend()
    plt.tight_layout()

    #Save file to save location
    if save:            
        #Try creating dir to save spline images to
        splines_save_dir = f'C:\\Users\\victo\\Programming\\l-ded_ml_models\\spline_plots\\plots_{degree}degrees_{knots}knots'
        try:
            os.mkdir(splines_save_dir)
        except:
            pass

        #Setup filename
        filename = splines_save_dir + f'\\NEW_spline_label_{int(np.round(label))}_{np.random.randint(low=0, high=1000000)}.jpg'

        #Save images
        plt.savefig(filename, bbox_inches='tight')

    #Show figure of splines or not
    if show:
        plt.show()
    else:
        plt.close()

def spline_plots(degree, knots, alpha=1e-3, save=False, show=False):
    bef_reg = pd.read_csv('C:\\Users\\victo\\Programming\\l-ded_ml_models\\files\\d_before_spline.csv', index_col=0)
    after_reg = bef_reg.copy(deep=True)
    
    #Loop over the dataframe for each label
    for label_name, reg_df in bef_reg.groupby(by='label'):
        spline_df = reg_df[~reg_df['local'].isnull()][['x pos', 'value', 'id for reg']]

        #define training points for spline
        x_train = spline_df['x pos'].to_numpy()
        y_train = spline_df['value'].to_numpy()

        #Reshape for input on pipeline
        X_train = x_train[:, np.newaxis]
        Y_train = y_train[:, np.newaxis]

        #Define and fit spline model on training data
        spline_model = make_pipeline(SplineTransformer(n_knots=knots, degree=degree), Ridge(alpha=alpha))
        spline_model.fit(X_train, Y_train)

        #Prediction
        x_pred = reg_df.loc[~reg_df['value'].isnull()]['x pos'].to_numpy()
        X_pred = x_pred[:, np.newaxis] #Reshape for prediction
        y_pred = spline_model.predict(X_pred) #Predict

        #plots
        x_plot =  np.linspace(0, 136, 280)
        X_plot = x_plot[:, np.newaxis]
        y_plot = spline_model.predict(X_plot)

        #Save plots
        plot_spline(degree, knots, X_plot, y_plot, x_train, y_train, label_name, save=save, show=show)

        #write to main dataframe based on the indexes and the spline predictions
        index_list = reg_df.loc[~reg_df['value'].isnull()].index.tolist()
        for index, reg_value in zip(index_list, y_pred.tolist()):
            after_reg.loc[index, 'reg value'] = reg_value
        if show:
            break
    


In [43]:
SPLINES = [
    (2,3),
    (3,4),
    (5,6),
    (7,8),
    (8,9),
    (9,10),

    #(1,10),
    (2,10),
]

In [45]:
def make_plots(dir_path):
    global eval_results_df
    eval_results_df = pd.read_csv(os.path.join(dir_path, 'eval_results_df.csv'), index_col=0)
    
    #save_true_vs_pred_plot(eval_results_df, dir_path, save=False)
    #save_mean_true_vs_pred_plot(eval_results_df, dir_path, save=True)
    save_width_over_x_plot(eval_results_df, dir_path, adjusted=True, save=True)
    #spline_plots(degree=8, knots=9, save=True, show=False)
    
    
dir_path = r'D:\Users\Victor\Área de Trabalho\train_history\7d8k\final\small_dense\densenet169'
make_plots(dir_path)

In [58]:
n_eval_results_df_path = [
    
]

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(3,2)

In [5]:
eval_results_df

,standoff distance,true,x pos,pred
0,3.50,0.946429,106.666,0.784385
1,5.15,0.696731,110.666,0.780644
2,3.45,0.869158,51.333,0.775954
3,2.50,0.783651,89.333,0.793609
4,5.20,0.704405,63.333,0.739052
...,...,...,...,...
2416,2.50,0.748129,118.666,0.794627
2417,3.20,0.800486,30.666,0.776456
2418,3.60,0.890105,81.333,0.785667
2419,5.20,0.658894,73.333,0.709672
